# Getting started

## Before you start

### Custom Dataset
If you want to run prompt optimization on your own dataset, follow these steps:

1. Create a folder.
1. Create a .txt file in the folder named "prompts.txt". It should contain 8-12 initial prompts from where you can start the optimization. Add line breaks between each of the prompts
1. Create two .txt files in another folder, which contain the dev set "dev.txt" and test set "test.txt" of your data points. Convert the classes of your file into integers. 
Make sure to seperate the input from the expected output with a tab!
1. Create a description.json file that contains a dictionary, specifying:
    - "seed": the folder in which you find the dev and test files
    - "init_prompts": the name of the .txt file pointing to the prompts
    - "description": A short description of your task, that is fed to the meta-llm in order to optimize the prompts. 
    (TIP: Include "The class mentioned first in the response of the LLM will be the prediction." in the description if this is how you evaluate the models responses)
    - "classes": A list of the names of the classes you are trying to predict

You can find examples of how this needs to be set up in our repo at data_sets/

## Installs

In [ ]:
! pip install promptolution

## Imports

In [1]:
from promptolution.helpers import run_experiment
from promptolution.config import Config

c:\Users\tzehl\Documents\programming\promptolution\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## set up llms, predictor, tasks and optimizer

In [2]:
token = open("../deepinfratoken.txt", "r").read()

In [25]:
config = Config(
    task_name="subj",
    ds_path="../data_sets/cls/subj/",
    n_steps=8,
    optimizer="evopromptde",
    meta_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    evaluation_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    downstream_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    api_token=token,
    prepend_exemplars=True,
    exemplar_selector="random_search",
    n_exemplars=3,
)

In [26]:
df = run_experiment(config)

In [5]:
df

,prompt,score
1,evaluate each sentence as either objective or ...,0.80
8,"As a linguist, analyze a statement from a movi...",0.80
3,identify whether the given sentence was expres...,0.65
5,Analyze the textual content of a given stateme...,0.65
9,determine the classification of each sentence ...,0.60
0,evaluate each statement as either subjective o...,0.50
2,Classify the sentence according to its subject...,0.40
6,"As a classifier, interpret phrases in movie re...",0.35
7,and\n\nshae is about to return to bed when she...,0.35
4,Analyze reviews and label them as subjective o...,0.30
